# Analisis inicial de datos con Spark

## Introduccion

Para la entrega del diseño inicial debemos tener explicatados en el documentos los siguiente pasos:

1- Clasificar el problema. Que tipo de problema tenemos? Regresion o clasificacion? Por que? <br>

2- Tenemos los datos. <br>
2.1 Hay que depurarlos. <br>
2.2 Detectar y eliminar anomalias <br>
2.3 Valores faltantes para ciertos atributos <br>

3- Explorar datos (no se procesan) <br>
3.1 Analizar la estructura <br>
3.2 Navegar datos <br>
3.3 Resumen estadistico. <br>
3.4 Visualizar datos <br>

4- Aplicar a los datos el algoritmo necesario <br>

5- Comunicar resultados <br>

Una de las herramientas que aprendimos y nos resulta sencilla es el Map-Reduce de Spark.<br>
En el caso de necesitar graficos podemos pedirle al Spark Context que nos devuelva los datos y luego se los pasamos a una libreria de ploteo de Python3.


## Desarrollo

In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')    